In [ ]:
#Pipeline imports
%matplotlib inline
%load_ext autoreload
%autoreload 2
import glob
import numpy as np
import matplotlib.pyplot as plt
import wirc_drp.wirc_object as wo
import wirc_drp.constants as constants
from wirc_drp.utils import calibration, spec_utils as su, image_utils as iu, source_utils as src
from wirc_drp.masks import *
import copy

from astropy.io import fits
from astropy.stats import sigma_clipped_stats

%load_ext autoreload
%autoreload 2

# Set target name and observation date

In [ ]:
target_name = 'SN2020oi'
obs_date = '20190119'

# Load sample images from the observation and run find source

In [ ]:
test_data = wo.wirc_data(wirc_object_filename= "/scr/data/Calibrated Files/20200119/Auto_Reduced/image0584_cal.fits")
bkg_image1 = wo.wirc_data(wirc_object_filename="/scr/data/Calibrated Files/20200119/Auto_Reduced/image0578_cal.fits")
# bkg_image2 = wo.wirc_data(wirc_object_filename="/scr/data/Calibrated Files/2020119/Auto_Reduced/image_cal.fits")

In [ ]:
# wo.wirc_data.find_sources_v2?

In [ ]:
test_data.bkg_image = bkg_image1.full_image*np.nanmedian(test_data.full_image)/np.nanmedian(bkg_image1.full_image)
test_data.cross_correlation_template = None
test_data.find_sources_v2(show_plots=True)


# Make a list of all observations

In [ ]:
def make_list(start, stop, path, prefix):
    file_list = [path+prefix+str(x).zfill(4)+'_cal.fits' for x in np.arange(start, stop+1)]
    return file_list


In [ ]:
data_path = '/scr/data/Calibrated Files/20200119/Auto_Reduced/'
prefix = 'image' #sometimes it's wirc

#Example here make a list of files from image0584.fits to image0659.fits from data_path
#If there's a gap in your observations, run two make_list commands and add the resulting lists together
filelist = make_list(584, 659, data_path, prefix)

In [ ]:
#Get Half Wave Plate Sequence

# Background subtraction 
Uncomment a cell that is most appropriate for you.

In [ ]:
# AB Dither: Observations are done with some AB dither sequence, likely in slit
bkg_sub = 'AB'
bkg_ind = src.find_best_background(filelist)

In [ ]:
# Background frame
bkg_sub = 'background_frame'
bkg_frame = "background_image.fits"

In [ ]:
# Fit background profile inside slit
bkg_sub = 'slit_background'

In [ ]:
# Shift and subtract
# This is simply using a shifted science image as background frame. 
bkg_sub = 'shift'


# Define source location

In [ ]:
#This is the location of the zeroth order (undispersed) point source
#If source is observed inside the slit, use x = 1020 and y = 1060
source_pos_x = 1020
source_pos_y = 1060

# Run spectral extraction on all files

In [ ]:
output_dir = "/scr/data/ManualReduction/calibrated/SN2020oi/20200119_ST/"
spec_cube = np.zeros((len(filelist),4,3,161))

for i,fname in enumerate(filelist):
    print("File {} of {}: {}".format(i+1,len(filelist),fname))
    tst_data = wo.wirc_data(wirc_object_filename=fname,verbose=False)
    filter_name = tst_data.filter_name

    tst_data.add_source(source_pos_x,source_pos_y, slit_pos = "slitless", update_w_chi2_shift = True, verbose = True)
    #Fit background, for slit data only
    if bkg_sub == 'slit_background':
        tst_data.generate_bkg(method='slit_background',verbose=True,plot=True,vmax=3000, fit_width = 3)
    #AB subtraction
    elif bkg_sub == 'AB'
        tst_data.generate_bkg(method='scaled_bkg',bkg_fns = filelist[bkg_ind[i]],bkg_by_quadrants = True)
    #Shift and subtract if background frame is not available
    elif bkg_sub == 'shift'
        tst_data.generate_bkg(method='shift_and_subtract',shift_dir='horizontal', bkg_sub_shift_size = 31)
    elif bkg_sub == 'background_frame'
        tst_data.generate_bkg(method='scaled_bkg',bkg_fns = [bkg_frame] ,bkg_by_quadrants = True)
        
    #Get cutout of the four spectral traces
    wp_source = tst_data.source_list[0]
    wp_source.get_cutouts(tst_data.full_image,tst_data.DQ_image,filter_name = filter_name,
                              bkg_image = tst_data.bkg_image,
                              replace_bad_pixels=True,method='median',sub_bar=False)
    wp_source.plot_cutouts(figsize=(10,6),vmax = 3000, plot_bkg_sub=False)
    wp_source.plot_cutouts(figsize=(10,6),plot_dq=True)
    plt.show()
    #Here run spectral extraction
    wp_source.extract_spectra(method='optimal_extraction'
                                verbose=True,plot_result=False,
                              plot_findTrace=False,plot_optimal_extraction=True,
                             spatial_sigma=3,bad_pix_masking =1, trace_angle = [-44.8, -46.3, -43.7, -44.6])
    spec_cube[i] = wp_source.trace_spectra
    tst_data.source_list.append(wp_source)
    tst_data.n_sources += 1
    tst_data.save_wirc_object(output_dir+fname.split("/")[-1])
    
    del tst_data
    wp_source.plot_trace_spectra(figsize=(9,4))
    
    plt.show()

# Save spectra and HWP as numpy files

In [ ]:
np.save('spec_cube_%s_%s_%s_%s.npy'%(target_name, obs_date, filter_name, bkg_sub), spec_cube)

In [ ]:
HWP_all = []
for i in filelist:
    HWP_all += [fits.getheader(i)['HWP_ANG']]
    
np.save('HWP_%s_%s_%s.npy'%(target_name, obs_date, filter_name), HWP_all)